In [ ]:
import torch
from torch import nn
from torch import optim
import torch.utils
import torch.utils.data
import torchvision.models as models

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision import models

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

In [ ]:
test = torch.load('/kaggle/input/serminar-khoa-hc/pytorch/model/1/best_model.pth')

In [ ]:
train_dataset_path = '/kaggle/input/photo-of-infected-leaves/Train_data'
test_dataset_path = '/kaggle/input/photo-of-infected-leaves/Test_data'

In [ ]:
def to_device():
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  return device

In [ ]:
to_device()

In [ ]:
def get_train_transform(mean, std):
    """
    Tạo đối tượng transforms.Compose cho dữ liệu huấn luyện.

    Parameters:
        mean (list): Giá trị trung bình của dữ liệu.
        std (list): Độ lệch chuẩn của dữ liệu.

    Returns:
        transforms.Compose: Đối tượng biến đổi dữ liệu cho dữ liệu huấn luyện.
    """
    train_transform = transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    ])
    return train_transform

def get_test_transform(mean, std):
    """
    Tạo đối tượng transforms.Compose cho dữ liệu kiểm tra.

    Parameters:
        mean (list): Giá trị trung bình của dữ liệu.
        std (list): Độ lệch chuẩn của dữ liệu.

    Returns:
        transforms.Compose: Đối tượng biến đổi dữ liệu cho dữ liệu kiểm tra.
    """
    test_transform = transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    ])
    return test_transform

# Sử dụng hàm để tạo các đối tượng biến đổi cho dữ liệu huấn luyện và kiểm tra
mean = [0.4363, 0.4328, 0.3291]
std = [0.2129, 0.2075, 0.2038]

train_transform = get_train_transform(mean, std)
test_transform = get_test_transform(mean, std)


In [ ]:
def show_transformed_images(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size = 6, shuffle=True)
    batch = next(iter(loader))
    images, labels = batch
    
    grid = torchvision.utils.make_grid(images, nrow=3)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(grid, (1,2,0)))
    print('labels: ', labels)

In [ ]:
temp_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transform)
show_transformed_images(temp_dataset)

In [ ]:
def create_data_loaders(train_dataset_path, test_dataset_path, train_transform, test_transform, batch_size=32):
    """
    Tạo các đối tượng DataLoader cho dữ liệu huấn luyện và kiểm tra.

    Parameters:
        train_dataset_path (str): Đường dẫn đến thư mục chứa dữ liệu huấn luyện.
        test_dataset_path (str): Đường dẫn đến thư mục chứa dữ liệu kiểm tra.
        train_transform (transforms.Compose): Đối tượng biến đổi dữ liệu cho dữ liệu huấn luyện.
        test_transform (transforms.Compose): Đối tượng biến đổi dữ liệu cho dữ liệu kiểm tra.
        batch_size (int, optional): Kích thước batch. Mặc định là 32.

    Returns:
        train_loader (torch.utils.data.DataLoader): DataLoader cho dữ liệu huấn luyện.
        test_loader (torch.utils.data.DataLoader): DataLoader cho dữ liệu kiểm tra.
    """
    # Tạo đối tượng ImageFolder từ đường dẫn dữ liệu huấn luyện và kiểm tra
    train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transform)
    test_dataset = torchvision.datasets.ImageFolder(root=test_dataset_path, transform=test_transform)
    
    # Tạo DataLoader cho dữ liệu huấn luyện và kiểm tra
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

train_loader, test_loader = create_data_loaders(train_dataset_path, test_dataset_path, train_transform, test_transform)


In [ ]:
def save_checkpoint(models, epoch, optimizer, best_acc):
    state = {
        'epoch' : epoch + 1,
        'model' : models.state_dict(),
        'best accuracy' : optimizer.state_dict(),
        'comments' : 'Verry coll model',
    }
    torch.save(state, '/kaggle/working/model_best_checkpoint.pth.tar')

In [ ]:
def evaluate_model_on_test_set(model: torch.nn.Module, 
                               test_loader: torch.utils.data.DataLoader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = to_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    epoch_acc = 100.0 * predicted_correctly_on_epoch / total
    print("    - Testing dataset. Got %d out of %d images correctly (%.3f%%)"
          % (predicted_correctly_on_epoch, total, epoch_acc))
    return epoch_acc
    

In [ ]:
accurecy_train = []
loss_train = []

In [ ]:
def train_nn(model: torch.nn.Module, 
             train_loader: torch.utils.data.DataLoader, 
             test_loader: torch.utils.data.DataLoader, 
             loss_fn: torch.nn.Module, 
             optimizer: torch.optim.Optimizer, 
             n_epochs):
    
    best_acc = 0
    device = to_device()
    
    for epoch in range(n_epochs):
        print("Epoch number %d" % (epoch + 1))
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            #reset đạo hàm
            optimizer.zero_grad()
            #output với trọng số hiện tại
            outputs = model(images)
            #lấy giá trị output theo nhãn
            _, predicted = torch.max(outputs.data, 1)
            #tính loss hiện tại
            loss = loss_fn(outputs, labels)
            #đạo hàm loss
            loss.backward()
            # SGD
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
            
        epoch_loss = running_loss/len(train_loader)
        loss_train.append(epoch_loss)
        epoch_acc = 100.00 * running_correct / total
        accurecy_train.append(running_correct / total)
        
        print("    - Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f"
              % (running_correct, total, epoch_acc, epoch_loss))
        
        test_dataset_acc = evaluate_model_on_test_set(model, test_loader)
        
        if test_dataset_acc > best_acc:
            best_acc = test_dataset_acc
            print("found better accuracy")
            save_checkpoint(model, epoch, optimizer, best_acc)
        
    print("Finished")
    return model

In [ ]:
def initialize_resnet18_model(num_classes, pretrained=True, lr=0.01, momentum=0.9, weight_decay=0.003):
    """
    Khởi tạo mô hình ResNet-18 cho bài toán phân loại với số lớp đầu ra được chỉ định.

    Parameters:
        num_classes (int): Số lớp đầu ra (số lớp phân loại).
        pretrained (bool, optional): Sử dụng mô hình được huấn luyện trước hay không. Mặc định là True.
        lr (float, optional): Tốc độ học của bộ tối ưu hóa. Mặc định là 0.01.
        momentum (float, optional): Tham số momentum của bộ tối ưu hóa SGD. Mặc định là 0.9.
        weight_decay (float, optional): Hệ số giảm trọng lượng của bộ tối ưu hóa. Mặc định là 0.003.

    Returns:
        model (torch.nn.Module): Mô hình ResNet-18 đã được khởi tạo.
        criterion (torch.nn.Module): Hàm mất mát (loss function).
        optimizer (torch.optim.Optimizer): Bộ tối ưu hóa được sử dụng cho huấn luyện.
    """
    # Khởi tạo mô hình ResNet-18
    model = models.resnet18(pretrained=pretrained)
    
    # Thay đổi lớp cuối cùng (fully connected layer) để phù hợp với số lớp đầu ra mới
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    device = to_device()
    model = model.to(device)
    # Định nghĩa hàm mất mát (cross entropy loss)
    criterion = nn.CrossEntropyLoss()
    
    # Khởi tạo bộ tối ưu hóa (SGD)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    
    return model, criterion, optimizer

# Sử dụng hàm để khởi tạo mô hình ResNet-18 và các thành phần liên quan
number_of_classes = 3
resnet18_model, loss_fn, optimizer = initialize_resnet18_model(num_classes=number_of_classes)


In [ ]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 150)

In [ ]:
plt.plot(accurecy_train, label = "Accuracy")

plt.title = "Biểu đồ accuracy"
plt.xlabel = "Accuracy"
plt.ylabel = "Epoch"

plt.legend()
plt.show

In [ ]:
plt.plot(loss_train, label = "Loss")

plt.title = "Biểu đồ loss"
plt.xlabel = "Loss"
plt.ylabel = "Epoch"

plt.legend()
plt.show

In [ ]:
# number_of_classes = 3
# resnet18_model, loss_fn, optimizer = initialize_resnet18_model(num_classes=number_of_classes, pretrained=True)


In [ ]:
# train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 20)

In [ ]:
checkpoint = torch.load('/kaggle/working/model_best_checkpoint.pth.tar')

In [ ]:
# print(checkpoint['epoch'])
# print(checkpoint['comments'])
# print(checkpoint['best accuracy'])

In [ ]:
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 3
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = to_device()
resnet18_model.load_state_dict(checkpoint['model'])

torch.save(resnet18_model, '/kaggle/working/best_model.pth')

In [ ]:
# !rm -rf /kaggle/working/*